In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d mehradaria/leukemia

 88% 97.0M/110M [00:01<00:00, 65.2MB/s]
100% 110M/110M [00:01<00:00, 81.1MB/s] 


In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/leukemia.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define hyperparameters
IMG_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 4
NUM_CLASSES = 4

# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(directory='/content/Original',
                                                    target_size=(IMG_SIZE, IMG_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    shuffle=True)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(directory='/content/drive/MyDrive/Leukemia/test',
                                                target_size=(IMG_SIZE, IMG_SIZE),
                                                batch_size=BATCH_SIZE,
                                                class_mode='categorical',
                                                shuffle=False)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(directory='/content/drive/MyDrive/Leukemia/test',
                                                  target_size=(IMG_SIZE, IMG_SIZE),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical',
                                                  shuffle=False)

# Define the model






Found 3256 images belonging to 4 classes.
Found 876 images belonging to 4 classes.
Found 876 images belonging to 4 classes.


In [6]:

# CUstom CNN  model
model = tf.keras.models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16

# Build the VGG16 model
base_model = VGG16(input_shape=(256, 256, 3), include_top=False)

# Freeze the weights of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.BatchNormalization()(x)
predictions = Dense(4, activation='softmax')(x)  # change activation to softmax and units to 4

# Create the VGG16 model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # change loss to categorical_crossentropy


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history=model.fit(train_generator, epochs=5, validation_data=val_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

Epoch 1/5
102/102 [==============================] - ETA: 0s - loss: 0.3764 - accuracy: 0.8768

In [ ]:
model.save("Leukemiavgg.h5")

In [ ]:
import cv2
from tensorflow.keras.models import load_model
test_img=cv2.imread('/content/Original/Pro/WBC-Malignant-Pro-190.jpg')
model1=load_model('Leukemiavgg.h5')

#Mapping result array with the main name list
test_img = cv2.resize(test_img,(256,256))
test_input = test_img.reshape((1,256,256,3))
result=model1.predict(test_input)
import numpy as np

# Assuming that the order of the classes is ['class1', 'class2', 'class3', 'class4']
class_names = ['class1', 'class2', 'class3', 'class4']

# Get the index of the class with the highest probability
predicted_class_index = np.argmax(result)

# Get the name of the predicted class
predicted_class_name = class_names[predicted_class_index]

# Print the predicted class name
print(predicted_class_name)


print(result)

1/1 [==============================] - 0s 291ms/step
class4
[[0.0000000e+00 0.0000000e+00 2.4929663e-11 1.0000000e+00]]


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Get predictions from the model
test_predictions = model.predict(test_generator)

# Convert predictions to class labels
test_pred_labels = np.argmax(test_predictions, axis=1)

# Get true labels from the generator
test_true_labels = test_generator.classes

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Generate classification report
report = classification_report(test_true_labels, test_pred_labels, target_names=class_labels)
print("Classification Report of Custom CNN Model")

print(report)


28/28 [==============================] - 4s 125ms/step
Classification Report of Custom CNN Model
              precision    recall  f1-score   support

      Benign       0.90      0.94      0.92       204
       Early       0.97      0.94      0.95       216
         Pre       0.98      0.93      0.96       240
         Pro       0.95      1.00      0.98       216

    accuracy                           0.95       876
   macro avg       0.95      0.95      0.95       876
weighted avg       0.95      0.95      0.95       876

